In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class Data_Validation_Config:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list     

In [5]:
from src.txt_sm.constants import *
from src.txt_sm.utils.common import read_yaml,create_dirs


class Configuration_Manager:
    def __init__(self,config_path=CONFIG_PATH,params_path=PARAMS_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
    
        create_dirs([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> Data_Validation_Config:
        config = self.config
        
        create_dirs([config.root_dir])
        data_validation_config = Data_Validation_Config(root_dir=config.root_dir,
                                                      STATUS_FILE=config.STATUS_FILE,
                                                      ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES)
        return data_validation_config

In [6]:
import urllib.request as request
import zipfile
from src.txt_sm.logging import logger
from src.txt_sm.utils.common import get_size


class DataValidation:
    def __init__(self,config:Data_Validation_Config ):
        self.config = config

    def initiate_data_validation(self):
        try:
            validation_status = False
            all_files = os.listdir(os.path.join('artifacts','data_ingestion','data'))
            
            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as file:
                        file.write(f'Validation status: {validation_status}')
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE,'w') as file:
                        file.write(f'Validation status: {validation_status}')
            
            if not validation_status:
                logger.info('Couldnt Validate all files')
            else:
                logger.info('Successfully Validated all files')
            
            return validation_status            
        
        except Exception as e:
            raise e